In [8]:
import pandas as pd
import matplotlib as plt
import numpy as np


df = pd.read_csv("Simonelli dataset - Cleaned.csv")

#store dates as dates rather than a string
df['First_proposed_date'] = pd.to_datetime(df['First_proposed_date'])
df['Signature_date'] = pd.to_datetime(df['Signature_date'])
df['Date_entered_force'] = pd.to_datetime(df['Date_entered_force'],errors="coerce")

### Defining functions

In [9]:
def Stats(name,df):

    median = df["Negotiation_length_days"].median()
    median = round(median)

    mean = df["Negotiation_length_days"].mean()
    mean = round(mean)
    
    std = df["Negotiation_length_days"].std()
    std = round(std)
    
    TenthPercentile = df["Negotiation_length_days"].quantile(q=0.1)
    TenthPercentile = round(TenthPercentile)
    
    count = df.shape[0]

    return(name, median, mean, std, TenthPercentile, count)

col_list = ['Name','Median', 'Mean', "Standard Deviation", "Tenth Percentile","Count"]

In [10]:
def Actors(df):

    df_NGO = df[(df['Proposer_type'] == 'NGO')]
    df_IGO = df[(df['Proposer_type'] == 'IGO')]
    df_single = df[(df['Proposer_type'] == 'single state')]
    df_multiple = df[(df['Proposer_type'].str.contains("multiple"))]
    df_joint = df[(df['Proposer_type'].str.contains("joint"))]
    #some treaties can go into multiple categories here
    #all treaties are in at least one category
    
    a = Stats("NGO",df_NGO)
    b = Stats("IGO",df_IGO)
    c = Stats("Single state",df_single)
    d = Stats("Multiple separate proposals",df_multiple)
    e = Stats("Joint proposal from multiple actors",df_joint)

    data = [a,b,c,d,e]
    output = pd.DataFrame(data, columns=col_list)
    return(output)

### Time for different kinds of treaty

In [11]:
#all treaties
a = Stats("Multilateral treaties from 1864",df)


#in total from 1945
df_post_45 = df[(df['First_proposed_date'] > "1945-01-01")]
b = Stats("Multilateral treaties after 1945",df_post_45)


#by security from 1864
df_not_security = df[(df['Non_security'] == 1)]
c = Stats("Multilateral non-security treaties from 1864",df_not_security)

df_security = df[(df['Non_security'] == 0)]
d = Stats("Multilateral security treaties from 1864",df_security)


#by security from 1945
df_45_not_security = df_post_45[(df_post_45['Non_security'] == 1)]
e = Stats("Multilateral non-security treaties from 1945",df_not_security)

df_45_security = df_post_45[(df_post_45['Non_security'] == 0)]
f = Stats("Multilateral security treaties from 1945",df_security)


#output
data = [a,b,c,d,e,f]

In [12]:
times = pd.DataFrame(data, columns=col_list)
times

,Name,Median,Mean,Standard Deviation,Tenth Percentile,Count
0,Multilateral treaties from 1864,592,900,1057,78,168
1,Multilateral treaties after 1945,759,1046,1029,262,127
2,Multilateral non-security treaties from 1864,660,900,943,144,81
3,Multilateral security treaties from 1864,531,900,1159,72,87
4,Multilateral non-security treaties from 1945,660,900,943,144,81
5,Multilateral security treaties from 1945,531,900,1159,72,87


### Time for post-1945 treaties by actor

In [13]:
Actors(df_post_45)

,Name,Median,Mean,Standard Deviation,Tenth Percentile,Count
0,NGO,1546,1662,761,839,12
1,IGO,564,919,860,180,52
2,Single state,742,1109,947,340,36
3,Multiple separate proposals,266,427,341,155,8
4,Joint proposal from multiple actors,714,1076,1591,366,20


### Time for post-1945 *security* treaty by actor

In [14]:
Actors(df_45_security)

,Name,Median,Mean,Standard Deviation,Tenth Percentile,Count
0,NGO,1204,1546,769,839,10
1,IGO,674,1161,1094,318,14
2,Single state,884,1265,1054,396,20
3,Multiple separate proposals,422,422,279,264,2
4,Joint proposal from multiple actors,620,1422,2325,401,9


### Time for post-1945 *not security* treaty by actor

In [15]:
Actors(df_45_not_security)

,Name,Median,Mean,Standard Deviation,Tenth Percentile,Count
0,NGO,2242,2242,499,1960,2
1,IGO,552,830,754,166,38
2,Single state,673,914,784,299,16
3,Multiple separate proposals,266,428,383,116,6
4,Joint proposal from multiple actors,730,793,535,325,11


### Time for post-1945 treaties, broken down by great power led

In [16]:
#proposed by great power related to security or not
df_great_power = df_post_45[(df_post_45['Majorpower_prop'] == 1)]
a1 = Stats("Great power led in general",df_great_power)

df_great_power_sec = df_great_power[(df_great_power['Non_security'] == 0)]
a2 = Stats("Great power led about security",df_great_power_sec)

df_great_power_not_sec = df_great_power[(df_great_power['Non_security'] == 1)]
a3 = Stats("Great power led not about security",df_great_power_not_sec)


#not proposed by great power
df_not_great_power = df_post_45[(df_post_45['Majorpower_prop'] == 0)]
b = Stats("NOT great power led",df_not_great_power)

times = pd.DataFrame((a1,a2,a3,b), columns=col_list)
times

#NB: These are all looking just from 1945

,Name,Median,Mean,Standard Deviation,Tenth Percentile,Count
0,Great power led in general,792,1174,1422,247,33
1,Great power led about security,759,1489,1770,407,19
2,Great power led not about security,802,747,546,134,14
3,NOT great power led,723,1001,856,274,94


## Lists for Guesstimate
https://www.getguesstimate.com/models/21452

In [17]:
#function to give CSV of all treaties in that category
def CSV(df):
    output = []
    
    for i in df["Negotiation_length_days"]:
        i = i/365
        output.append(i)
    
    return(output)

#function to give CSV of treaties in that category between 10th and 30th percentile
def Percentile(CSV):
    output = []
    
    ten = np.percentile(CSV,10)
    thirty = np.percentile(CSV,30)
    
    for i in CSV:
        if ((i>ten) and (i<thirty)):
            output.append(i)
    
    return(output)

In [18]:
#ALL multilateral treaties since 1945
CSV(df_post_45)

[1.610958904109589,
 2.2986301369863016,
 2.2986301369863016,
 2.2986301369863016,
 2.2986301369863016,
 1.4547945205479451,
 11.317808219178081,
 4.764383561643836,
 0.6164383561643836,
 2.2164383561643834,
 2.873972602739726,
 1.904109589041096,
 3.6794520547945204,
 2.0794520547945203,
 1.3068493150684932,
 1.3123287671232877,
 3.2986301369863016,
 3.2986301369863016,
 4.506849315068493,
 6.641095890410959,
 6.641095890410959,
 6.641095890410959,
 6.641095890410959,
 1.4109589041095891,
 1.9671232876712328,
 0.9424657534246575,
 8.931506849315069,
 2.2739726027397262,
 20.81095890410959,
 10.441095890410958,
 2.115068493150685,
 2.7643835616438355,
 0.9643835616438357,
 1.1205479452054794,
 5.147945205479452,
 1.6986301369863013,
 1.621917808219178,
 4.046575342465753,
 3.115068493150685,
 7.227397260273973,
 2.084931506849315,
 0.39452054794520547,
 0.6767123287671233,
 1.6191780821917807,
 0.2191780821917808,
 0.8164383561643835,
 0.7808219178082192,
 0.8383561643835616,
 0.589041

In [19]:
#Multilateral treaties since 1945 [10th to 30th percentile]   
post45 = CSV(df_post_45)
Percentile(post45)

[1.3068493150684932,
 0.9424657534246575,
 0.9643835616438357,
 1.1205479452054794,
 0.8164383561643835,
 0.7808219178082192,
 0.8383561643835616,
 1.0383561643835617,
 0.7671232876712328,
 0.8712328767123287,
 1.2082191780821918,
 0.9616438356164384,
 0.8493150684931506,
 0.9205479452054794,
 1.0136986301369864,
 1.1123287671232878,
 1.0986301369863014,
 0.7452054794520548,
 1.0794520547945206,
 0.9232876712328767,
 1.1726027397260275,
 1.0082191780821919,
 1.0082191780821919,
 0.863013698630137,
 0.8904109589041096]

In [20]:
#ALL multilateral treaties that were proposed by a great power after 1945 and that are about security
CSV(df_great_power_sec)

[11.317808219178081,
 4.764383561643836,
 0.6164383561643836,
 2.873972602739726,
 1.904109589041096,
 3.6794520547945204,
 2.0794520547945203,
 1.3068493150684932,
 1.9671232876712328,
 2.2739726027397262,
 20.81095890410959,
 1.1205479452054794,
 1.6986301369863013,
 1.621917808219178,
 3.115068493150685,
 5.087671232876712,
 1.9863013698630136,
 1.0986301369863014,
 8.210958904109589]

In [21]:
#Multilateral treaties that were proposed by a great power after 1945 
#and that are about security [10th to 30th percentile]

greatpower = CSV(df_great_power_sec)
Percentile(greatpower)

[1.3068493150684932, 1.1205479452054794, 1.6986301369863013, 1.621917808219178]

## Exporting subsets to CSV

In [22]:
exportDF = df_great_power_sec.loc[:,["Title","First_proposed_date","Date_entered_force","How_many_states","First_proposer"]]
exportDF.to_csv("post45_security_greatpower.csv")

## Graphs

In [23]:
def Histogram(df, title):
    hist = df["Negotiation_length_days"].hist(bins=60,xlabelsize=None)
    hist.set_title(title)
    hist.set_xlabel("Duration length (days)")
    hist.set_ylabel("Count")
    return(hist)

In [24]:
# Histogram(df, "Entire dataset")

In [25]:
# Histogram(df_post_45, "All agreements after 1945")